In [1]:
import pandas as pd
import glob
import os

In [69]:
path = r"../tab2/new_tweets"
files = glob.glob(os.path.join(path , "*.csv"))
files
df_test = pd.read_csv("../tab2/new_tweets/Airbnb_2019-07-01_2019-08-01.csv", sep=";")
df_tweets = pd.DataFrame(columns=df_test.columns)
df_tweets

,Unnamed: 0,id,url,date,content,outlinks,username,language,sentiment_neg,sentiment_neu,sentiment_pos


In [70]:
for i in files:
    df_file = pd.read_csv(i, sep=";")
    # df_file["Unnamed: 0"] = str.split(str.rsplit(i, sep="\\", maxsplit=1)[1], sep="_", maxsplit=1)[0]
    df_tweets = pd.concat([df_tweets, df_file], axis=0)

In [35]:
# df_tweets = pd.read_csv("../test/aggregated_twitter_data.csv",delimiter=";")


In [71]:
# Select only relevant columns
columns = ["date", "sentiment_neg", "sentiment_neu", "sentiment_pos"]
df_tweets = df_tweets[[col for col in df_tweets.columns if col in columns]]

# Replace , with . and convert to float
df_tweets["sentiment_neg"] = df_tweets["sentiment_neg"].str.replace(",", ".").astype(float)
df_tweets["sentiment_pos"] = df_tweets["sentiment_pos"].str.replace(",", ".").astype(float)
df_tweets["sentiment_neu"] = df_tweets["sentiment_neu"].str.replace(",", ".").astype(float)

# Set date format
df_tweets["date"] = pd.to_datetime(df_tweets["date"], format="%Y-%m-%d")
df_tweets["date"] = df_tweets["date"].dt.strftime("%Y-%m-%d")

# Set brand name and drop rows with missing values
df_tweets["brand"] = "Airbnb"

df_tweets_all = df_tweets
# Remove rows with no sentiment
df_tweets = df_tweets.dropna()
df_tweets_all = df_tweets_all.fillna(0)




df_tweets

,date,sentiment_neg,sentiment_neu,sentiment_pos,brand
0,2019-07-31,0.000,0.629,0.371,Airbnb
1,2020-11-12,0.000,0.850,0.150,Airbnb
4,2019-07-31,0.059,0.838,0.103,Airbnb
5,2019-07-31,0.000,0.663,0.337,Airbnb
9,2019-07-31,0.000,0.900,0.100,Airbnb
...,...,...,...,...,...
7447,2020-10-01,0.000,1.000,0.000,Airbnb
7448,2020-10-01,0.099,0.785,0.116,Airbnb
7449,2020-10-01,0.000,1.000,0.000,Airbnb
7451,2020-10-01,0.184,0.733,0.083,Airbnb


In [25]:
# Calculate overall sentiment
df_sentiment_overall = df_tweets[["brand","sentiment_neg","sentiment_pos"]].groupby("brand").mean().reset_index()
df_sentiment_overall["sentiment_neg_perc"] = ((df_sentiment_overall["sentiment_neg"] / (df_sentiment_overall["sentiment_neg"] + df_sentiment_overall["sentiment_pos"]))*100).round(2)
df_sentiment_overall["sentiment_pos_perc"] = ((df_sentiment_overall["sentiment_pos"] / (df_sentiment_overall["sentiment_neg"] + df_sentiment_overall["sentiment_pos"]))*100).round(2)
df_sentiment_overall

,brand,sentiment_neg,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc
0,Airbnb,0.03147,0.114608,21.54,78.46


In [110]:
# Calculate overall sentiment
df_sentiment_overall = df_tweets[["brand","sentiment_neg","sentiment_pos"]].groupby("brand").mean().reset_index()
# df_sentiment_overall["sentiment_neg_perc"] = ((df_sentiment_overall["sentiment_neg"] / (df_sentiment_overall["sentiment_neg"] + df_sentiment_overall["sentiment_pos"]))*100).round(2)
# df_sentiment_overall["sentiment_pos_perc"] = ((df_sentiment_overall["sentiment_pos"] / (df_sentiment_overall["sentiment_neg"] + df_sentiment_overall["sentiment_pos"]))*100).round(2)


In [111]:
def calculate_sentiment_percentages(df, neg_col, pos_col):
    total = df[neg_col] + df[pos_col]
    df[f"{neg_col}_perc"] = ((df[neg_col] / total) * 100).round(2)
    df[f"{pos_col}_perc"] = ((df[pos_col] / total) * 100).round(2)
    return df
df_sentiment_overall = calculate_sentiment_percentages(df_sentiment_overall, "sentiment_neg", "sentiment_pos")
df_sentiment_overall


,brand,sentiment_neg,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc
0,Airbnb,0.03147,0.114608,21.54,78.46


In [108]:
df_sentiment_overall

In [47]:
# Calculate daily polarity
df_sentiment_day = df_tweets.groupby(["brand", "date"]).mean().reset_index()
df_sentiment_day["sentiment_neg_perc"] = ((df_sentiment_day["sentiment_neg"] / (df_sentiment_day["sentiment_neg"] + df_sentiment_day["sentiment_pos"]))*100).round(2)
df_sentiment_day["sentiment_pos_perc"] = ((df_sentiment_day["sentiment_pos"] / (df_sentiment_day["sentiment_neg"] + df_sentiment_day["sentiment_pos"]))*100).round(2)
df_sentiment_day["polarity"] = df_sentiment_day["sentiment_pos_perc"] - df_sentiment_day["sentiment_neg_perc"]

df_sentiment_day.rename(columns={"date":"Date"}, inplace=True)
df_sentiment_day = df_sentiment_day.dropna()
df_sentiment_day

,brand,Date,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc,polarity
0,Airbnb,2018-08-13,0.063000,0.654133,0.282867,18.22,81.78,63.56
1,Airbnb,2018-08-16,0.072000,0.656867,0.271133,20.98,79.02,58.04
6,Airbnb,2019-05-02,0.229130,0.770870,0.000000,100.00,0.00,-100.00
7,Airbnb,2019-07-01,0.026629,0.884532,0.088835,23.06,76.94,53.88
8,Airbnb,2019-07-02,0.024659,0.873564,0.101766,19.51,80.49,60.98
...,...,...,...,...,...,...,...,...
495,Airbnb,2020-11-02,0.042000,0.850000,0.108000,28.00,72.00,44.00
496,Airbnb,2020-11-12,0.000000,0.850000,0.150000,0.00,100.00,100.00
497,Airbnb,2020-11-17,0.007077,0.958308,0.034615,16.97,83.03,66.06
498,Airbnb,2020-11-18,0.000000,0.687200,0.312800,0.00,100.00,100.00


In [12]:
# Calculate weekly polarity
df_tweets["date"] = pd.to_datetime(df_tweets["date"], format="%Y-%m-%d")
df_sentiment_week = df_tweets.groupby(["brand", pd.Grouper(key="date", freq="W-SUN")]).mean().reset_index()
df_sentiment_week.rename(columns={"date":"Date"}, inplace=True)
df_sentiment_week["Date"] -= pd.to_timedelta(6, unit="d")
df_sentiment_week["sentiment_neg_perc"] = ((df_sentiment_week["sentiment_neg"] / (df_sentiment_week["sentiment_neg"] + df_sentiment_week["sentiment_pos"]))*100).round(2)
df_sentiment_week["sentiment_pos_perc"] = ((df_sentiment_week["sentiment_pos"] / (df_sentiment_week["sentiment_neg"] + df_sentiment_week["sentiment_pos"]))*100).round(2)
df_sentiment_week["polarity"] = df_sentiment_week["sentiment_pos_perc"] - df_sentiment_week["sentiment_neg_perc"]
df_sentiment_week.dropna(inplace=True)
df_sentiment_week

In [96]:
# Calculate monthly polarity
df_sentiment_month = df_tweets.groupby(["brand", pd.Grouper(key="date", freq="M")]).mean().reset_index()
df_sentiment_month.rename(columns={"date":"Date"}, inplace=True)
df_sentiment_month["sentiment_neg_perc"] = ((df_sentiment_month["sentiment_neg"] / (df_sentiment_month["sentiment_neg"] + df_sentiment_month["sentiment_pos"]))*100).round(2)
df_sentiment_month["sentiment_pos_perc"] = ((df_sentiment_month["sentiment_pos"] / (df_sentiment_month["sentiment_neg"] + df_sentiment_month["sentiment_pos"]))*100).round(2)
df_sentiment_month["polarity"] = df_sentiment_month["sentiment_pos_perc"] - df_sentiment_month["sentiment_neg_perc"]
df_sentiment_month.dropna(inplace=True)
df_sentiment_month

,brand,Date,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc,polarity
0,Airbnb,2018-08-31,0.068143,0.655695,0.276162,19.79,80.21,60.42
3,Airbnb,2019-05-31,0.219583,0.780417,0.000000,100.00,0.00,-100.00
4,Airbnb,2019-07-31,0.025094,0.858716,0.116186,17.76,82.24,64.48
5,Airbnb,2019-08-31,0.026185,0.853972,0.119842,17.93,82.07,64.14
6,Airbnb,2019-09-30,0.023798,0.853547,0.122655,16.25,83.75,67.50
7,Airbnb,2019-10-31,0.027102,0.854345,0.118554,18.61,81.39,62.78
8,Airbnb,2019-11-30,0.046943,0.837386,0.115667,28.87,71.13,42.26
9,Airbnb,2019-12-31,0.023386,0.854222,0.122389,16.04,83.96,67.92
10,Airbnb,2020-01-31,0.020500,0.868243,0.111256,15.56,84.44,68.88
11,Airbnb,2020-02-29,0.027801,0.867298,0.104900,20.95,79.05,58.10


In [56]:
# Calculate daily tweet count
df_tweets_count_daily = df_tweets_all[["brand",	"date",	"sentiment_neg"]].groupby(["brand", "date"]).count().reset_index()
df_tweets_count_daily.rename(columns={"date":"Date", "sentiment_neg":"tweet_count"}, inplace=True)
df_tweets_count_daily

,brand,Date,tweet_count
0,Airbnb,2018-08-13,90
1,Airbnb,2018-08-16,120
2,Airbnb,2018-11-15,27
3,Airbnb,2018-11-27,4
4,Airbnb,2019-01-06,6
...,...,...,...
496,Airbnb,2020-11-02,3
497,Airbnb,2020-11-12,1034
498,Airbnb,2020-11-17,16
499,Airbnb,2020-11-18,5


In [75]:
df_tweets_all["date"] = pd.to_datetime(df_tweets_all["date"], format="%Y-%m-%d")
df_tweets_count_weekly = df_tweets_all[["brand","date",	"sentiment_neg"]].groupby(["brand", pd.Grouper(key="date", freq="W-SUN")]).count().reset_index()
df_tweets_count_weekly.rename(columns={"date":"Date", "sentiment_neg":"tweet_count"}, inplace=True)
df_tweets_count_weekly["Date"] -= pd.to_timedelta(6, unit="d")
df_tweets_count_weekly

,brand,Date,tweet_count
0,Airbnb,2018-08-13,210
1,Airbnb,2018-11-12,27
2,Airbnb,2018-11-26,4
3,Airbnb,2018-12-31,6
4,Airbnb,2019-04-22,68
...,...,...,...
74,Airbnb,2020-10-19,1711
75,Airbnb,2020-10-26,1172
76,Airbnb,2020-11-02,3
77,Airbnb,2020-11-09,1034


In [77]:
# pd.Grouper(key="date", freq="W-SUN") creates 0 values for weeks with no tweets!!!
df_tweets_all["date"] = pd.to_datetime(df_tweets_all["date"], format="%Y-%m-%d")
df_tweets_count_weekly = df_tweets_all[["brand","date",	"sentiment_neg"]].groupby(pd.Grouper(key="date", freq="W-SUN")).count().reset_index()
df_tweets_count_weekly.rename(columns={"date":"Date", "sentiment_neg":"tweet_count"}, inplace=True)
df_tweets_count_weekly["Date"] -= pd.to_timedelta(6, unit="d")
df_tweets_count_weekly

,Date,brand,tweet_count
0,2018-08-13,210,210
1,2018-08-20,0,0
2,2018-08-27,0,0
3,2018-09-03,0,0
4,2018-09-10,0,0
...,...,...,...
114,2020-10-19,1711,1711
115,2020-10-26,1172,1172
116,2020-11-02,3,3
117,2020-11-09,1034,1034


In [79]:
filename = "Apple_tweets.csv"
brand = str(filename.split("_")[0])
type = str(filename.split("_")[1].split(".")[0])
brand

'Apple'

In [83]:
path = r"../tab2/new_YouGov"
files = glob.glob(os.path.join(path , "*.csv"))
files
df_yougov = pd.read_csv("../tab2/new_YouGov/YouGov_Gesamt.csv", sep=";")
# df_yougov = pd.DataFrame(columns=df_test.columns)
# df_yougov

In [85]:
for brand in df_yougov["Brand"].unique():
    df_yougov.loc[df_yougov["Brand"] == brand, "Date"] = pd.date_range(
        start="2019-07-01", end="2021-02-28", freq = "D"
    )

In [88]:
df_yougov["Date"] = pd.to_datetime(df_yougov["Date"], format="%Y-%m-%d")

In [97]:
df_yougov["Date"] = pd.to_datetime(df_yougov["Date"], format="%Y-%m-%d")
df_yougov_weekly = df_yougov.groupby(["Brand", pd.Grouper(key="Date", freq="W-SUN")]).mean().reset_index().round(1)
df_yougov_weekly["Date"] -= pd.to_timedelta(6, unit="d")
df_yougov_weekly

,Brand,Date,Buzz,Impression,Quality,Value,Reputation,Satisfaction,Recommend,Awareness,Attention,Ad Awareness,WOM Exposure,Consideration,Purchase Intent,Current Customer,Former Customer
0,Airbnb,2019-07-01,7.1,4.1,4.1,8.2,7.2,8.7,5.6,56.4,16.7,7.2,14.7,16.3,8.2,7.2,5.5
1,Airbnb,2019-07-08,0.8,7.7,5.6,9.9,4.7,5.7,7.4,55.6,12.3,8.3,10.1,15.5,5.6,5.6,6.5
2,Airbnb,2019-07-15,3.7,7.5,6.6,10.4,9.2,7.8,8.2,53.1,12.6,8.1,10.7,13.6,6.1,6.3,5.3
3,Airbnb,2019-07-22,3.3,7.5,9.2,11.0,9.5,10.2,10.2,62.0,15.5,7.4,9.4,18.7,9.0,6.6,8.1
4,Airbnb,2019-07-29,8.1,15.4,13.9,15.6,11.3,11.9,14.7,61.8,16.6,11.3,15.2,19.9,8.9,11.2,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,Starbucks,2021-01-25,11.7,17.5,27.7,-11.8,15.1,22.2,14.7,90.0,29.0,27.6,21.8,25.4,13.2,19.8,40.8
605,Starbucks,2021-02-01,12.3,22.2,28.8,-11.4,18.3,20.2,18.6,88.3,30.5,24.7,21.9,29.9,15.1,21.3,40.1
606,Starbucks,2021-02-08,8.9,15.5,23.3,-18.4,11.8,15.5,7.9,90.8,25.5,24.3,18.4,23.9,12.2,18.2,43.4
607,Starbucks,2021-02-15,12.3,19.6,29.2,-13.5,11.9,24.2,14.5,90.0,26.4,22.1,22.7,29.4,14.2,19.6,41.3


In [98]:
df_yougov_monthly = df_yougov.groupby(["Brand", pd.Grouper(key="Date", freq="M")]).mean().reset_index().round(1)
df_yougov_monthly

,Brand,Date,Buzz,Impression,Quality,Value,Reputation,Satisfaction,Recommend,Awareness,Attention,Ad Awareness,WOM Exposure,Consideration,Purchase Intent,Current Customer,Former Customer
0,Airbnb,2019-07-31,4.2,7.5,7.3,10.2,8.4,8.3,8.5,57.8,14.8,8.6,11.6,16.3,7.2,7.0,6.0
1,Airbnb,2019-08-31,3.8,11.0,8.3,12.6,7.9,9.9,10.3,61.1,13.8,8.6,12.5,18.5,8.6,8.7,6.0
2,Airbnb,2019-09-30,4.6,10.3,7.2,12.2,8.4,11.0,10.0,61.6,15.5,10.6,12.6,17.9,8.6,9.0,7.2
3,Airbnb,2019-10-31,3.9,7.4,5.6,10.2,6.0,9.9,8.8,59.8,13.8,9.3,11.0,16.6,7.5,8.6,6.6
4,Airbnb,2019-11-30,2.1,9.3,7.4,11.6,7.2,11.3,9.7,63.6,17.8,11.1,13.0,18.5,8.3,9.5,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Starbucks,2020-10-31,14.1,19.9,28.8,-12.9,13.0,24.2,16.0,90.4,30.4,26.3,24.6,29.9,15.7,21.1,40.8
136,Starbucks,2020-11-30,9.8,17.5,26.7,-12.8,13.2,20.1,12.5,89.5,26.5,24.1,21.7,26.2,13.7,19.0,42.5
137,Starbucks,2020-12-31,12.3,19.4,28.0,-13.7,12.9,24.1,13.7,91.9,28.3,28.1,24.7,28.3,14.5,20.2,43.4
138,Starbucks,2021-01-31,11.9,19.3,27.8,-13.8,14.2,22.4,14.2,90.6,27.4,25.3,22.4,27.5,14.0,19.5,42.2


In [100]:
path = r"../tab2/new_stocktwits"
files = glob.glob(os.path.join(path , "*.csv"))
df_stocktwits = pd.read_csv("../tab2/new_stocktwits/Nike_stocktwits.csv", sep=";")
df_stocktwits

,date,trend,replies,likes,caption
0,"3/1/21, 11:23 PM",NaN,0,0,$NKE NEW ARTICLE : Nike's general manager of N...
1,"3/1/21, 11:03 PM",Bullish,1,0,$NKE Loving this stock.
2,"3/1/21, 11:02 PM",NaN,0,0,Your daily News digest for NIKE $NKE wsfriend....
3,"3/1/21, 10:51 PM",NaN,0,0,"$NKE NIKE, Inc. Announces Senior Leadership Ch..."
4,"3/1/21, 10:37 PM",NaN,0,0,"$NKE NEW ARTICLE : NIKE, Inc. Announces Senior..."
...,...,...,...,...,...
14719,"2/17/21, 10:10 PM",NaN,1,0,$NKE what are your price prediction for nke in...
14720,"12/22/20, 10:09 AM",NaN,0,0,Dow Jones Futures: House OKs Stimulus Deal Aft...
14721,"12/21/20, 04:25 PM",Bullish,0,0,$NKE NIKES ON MY FEET facebook.com/groups/5557...
14722,"9/23/20, 11:40 PM",NaN,0,0,Just put a $125 put on $NKE right when the ma...


In [101]:
df_stocktwits["brand"] = "Nike"

In [106]:
df_stocktwits["bullish"] = (df_stocktwits["trend"] == "Bullish").astype(int)
df_stocktwits["bearish"] = (df_stocktwits["trend"] == "Bearish").astype(int)
df_stocktwits.rename(columns={"date":"Date"}, inplace=True)
df_stocktwits["Date"] = pd.to_datetime(df_stocktwits["Date"])
df_stocktwits["Date"] = df_stocktwits["Date"].dt.date
df_stocktwits

,Date,trend,replies,likes,caption,brand,bullish,bearish
0,2021-03-01,NaN,0,0,$NKE NEW ARTICLE : Nike's general manager of N...,Nike,0,0
1,2021-03-01,Bullish,1,0,$NKE Loving this stock.,Nike,1,0
2,2021-03-01,NaN,0,0,Your daily News digest for NIKE $NKE wsfriend....,Nike,0,0
3,2021-03-01,NaN,0,0,"$NKE NIKE, Inc. Announces Senior Leadership Ch...",Nike,0,0
4,2021-03-01,NaN,0,0,"$NKE NEW ARTICLE : NIKE, Inc. Announces Senior...",Nike,0,0
...,...,...,...,...,...,...,...,...
14719,2021-02-17,NaN,1,0,$NKE what are your price prediction for nke in...,Nike,0,0
14720,2020-12-22,NaN,0,0,Dow Jones Futures: House OKs Stimulus Deal Aft...,Nike,0,0
14721,2020-12-21,Bullish,0,0,$NKE NIKES ON MY FEET facebook.com/groups/5557...,Nike,1,0
14722,2020-09-23,NaN,0,0,Just put a $125 put on $NKE right when the ma...,Nike,0,0


In [107]:
# Select only relevant columns
columns = ["Date", "brand", "bullish", "bearish"]
df_stocktwits = df_stocktwits[[col for col in df_stocktwits.columns if col in columns]]
df_stocktwits

,Date,brand,bullish,bearish
0,2021-03-01,Nike,0,0
1,2021-03-01,Nike,1,0
2,2021-03-01,Nike,0,0
3,2021-03-01,Nike,0,0
4,2021-03-01,Nike,0,0
...,...,...,...,...
14719,2021-02-17,Nike,0,0
14720,2020-12-22,Nike,0,0
14721,2020-12-21,Nike,1,0
14722,2020-09-23,Nike,0,0


In [108]:
df_stocktwits_daily = df_stocktwits[["brand","bullish","bearish","Date"]].groupby(["brand","Date"]).sum().reset_index()
df_stocktwits_daily["polarity"] = (df_stocktwits_daily["bullish"] - df_stocktwits_daily["bearish"])/(df_stocktwits_daily["bullish"] + df_stocktwits_daily["bearish"])*100
df_stocktwits_daily = df_stocktwits_daily.dropna()
df_stocktwits_daily["polarity"] = df_stocktwits_daily["polarity"].round(2)
df_stocktwits_daily

,brand,Date,bullish,bearish,polarity
0,Nike,2020-04-09,9,4,38.46
1,Nike,2020-04-10,0,1,-100.00
3,Nike,2020-04-12,1,1,0.00
4,Nike,2020-04-13,4,0,100.00
5,Nike,2020-04-14,11,1,83.33
...,...,...,...,...,...
377,Nike,2021-04-21,16,16,0.00
378,Nike,2021-04-22,4,20,-66.67
379,Nike,2021-04-23,10,6,25.00
380,Nike,2021-04-24,2,0,100.00


In [114]:
df_stocktwits["Date"] = pd.to_datetime(df_stocktwits["Date"])
df_stocktwits_weekly = df_stocktwits.groupby(["brand", pd.Grouper(key="Date", freq="W-SUN")]).sum().reset_index()
df_stocktwits_weekly["Date"] -= pd.to_timedelta(6, unit="d")
df_stocktwits_weekly["polarity"] = (df_stocktwits_weekly["bullish"] - df_stocktwits_weekly["bearish"])/(df_stocktwits_weekly["bullish"] + df_stocktwits_weekly["bearish"])*100
df_stocktwits_weekly["polarity"] = df_stocktwits_weekly["polarity"].round(2)
df_stocktwits_weekly

C:\Users\Toni\AppData\Local\Temp\ipykernel_2936\27935181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stocktwits["Date"] = pd.to_datetime(df_stocktwits["Date"])


,brand,Date,bullish,bearish,polarity
0,Nike,2020-04-06,10,6,25.00
1,Nike,2020-04-13,26,18,18.18
2,Nike,2020-04-20,11,9,10.00
3,Nike,2020-04-27,15,16,-3.23
4,Nike,2020-05-04,16,12,14.29
5,Nike,2020-05-11,30,19,22.45
6,Nike,2020-05-18,36,7,67.44
7,Nike,2020-05-25,33,28,8.20
8,Nike,2020-06-01,47,21,38.24
9,Nike,2020-06-08,27,5,68.75


In [116]:
df_stocktiwts_monthly = df_stocktwits.groupby(["brand", pd.Grouper(key="Date", freq="M")]).sum().reset_index()
df_stocktiwts_monthly["polarity"] = (df_stocktiwts_monthly["bullish"] - df_stocktiwts_monthly["bearish"])/(df_stocktiwts_monthly["bullish"] + df_stocktiwts_monthly["bearish"])*100
df_stocktiwts_monthly["polarity"] = df_stocktiwts_monthly["polarity"].round(2)
df_stocktiwts_monthly

,brand,Date,bullish,bearish,polarity
0,Nike,2020-04-30,55,40,15.79
1,Nike,2020-05-31,122,75,23.86
2,Nike,2020-06-30,423,378,5.62
3,Nike,2020-07-31,105,97,3.96
4,Nike,2020-08-31,190,55,55.10
5,Nike,2020-09-30,667,509,13.44
6,Nike,2020-10-31,134,78,26.42
7,Nike,2020-11-30,87,17,67.31
8,Nike,2020-12-31,559,92,71.74
9,Nike,2021-01-31,170,22,77.08


In [117]:
# Count of stocktwits per day
df_stocktwits_daily_count= df_stocktwits[["brand","bullish","bearish","Date"]].groupby(["brand","Date"]).count().reset_index()
df_stocktwits_daily_count

,brand,Date,bullish,bearish
0,Nike,2020-04-09,19,19
1,Nike,2020-04-10,13,13
2,Nike,2020-04-11,9,9
3,Nike,2020-04-12,9,9
4,Nike,2020-04-13,21,21
...,...,...,...,...
377,Nike,2021-04-21,69,69
378,Nike,2021-04-22,52,52
379,Nike,2021-04-23,60,60
380,Nike,2021-04-24,9,9


In [118]:
# Count of stocktwits per week
df_stocktwits_weekly_count = df_stocktwits.groupby(["brand", pd.Grouper(key="Date", freq="W-SUN")]).count().reset_index()
df_stocktwits_weekly_count["Date"] -= pd.to_timedelta(6, unit="d")
df_stocktwits_weekly_count

,brand,Date,bullish,bearish
0,Nike,2020-04-06,50,50
1,Nike,2020-04-13,157,157
2,Nike,2020-04-20,88,88
3,Nike,2020-04-27,99,99
4,Nike,2020-05-04,86,86
5,Nike,2020-05-11,162,162
6,Nike,2020-05-18,128,128
7,Nike,2020-05-25,196,196
8,Nike,2020-06-01,223,223
9,Nike,2020-06-08,121,121


In [5]:
import os
filename = "Apple_tweets.csv"
brand, data_source = filename.split('_')
data_source = os.path.splitext(data_source)[0]

In [6]:
data_source

'tweets'

In [58]:
def process_csv(contents, filename, date):
    date = "2021-03-01"
    if filename is not None:
            # Check if the file is a CSV
        if not filename.endswith('.csv'):
                return 'Please upload a CSV file'
            
        # Extract the brand and data source from the filename
        brand, data_source = filename.split('_')
        data_source = os.path.splitext(data_source)[0]
        
        directory = f"data/tab2/new_{data_source}/{brand}/"
        if data_source == "tweets":
            print(directory)
            # directory = f"data/tab2/new_{data_source}/{brand}/"
            if not os.path.exists(directory):
                print(directory)
                os.makedirs(directory)
            # Insert the date with an underscore after "tweets" in the filename
            new_filename = f"{brand}_tweets_{date}.csv"
            # Write the contents of the file to the specified directory
            with open(os.path.join(directory, new_filename), 'wb') as fp:
                fp.write(contents.encode('utf-8'))
    
    return None

In [59]:
process_csv("test", "Airbnb_tweets.csv", "2021-03-01")
# 

data/tab2/new_tweets/Airbnb/
data/tab2/new_tweets/Airbnb/


In [2]:
import os
current_directory = os.getcwd()
print(current_directory)

c:\Programming\Brand Perception Dashboard\src\data\notebooks


In [9]:
df_test = pd.read_csv("Airbnb_2019-08-01_2019-09-01.csv",delimiter=";")
columns = ["date", "sentiment_neg", "sentiment_neu", "sentiment_pos"]
df_test = df_test[[col for col in df_test.columns if col in columns]]
df_test.to_csv("Airbnb_tweets.csv", index=False)

In [10]:
df_test

,date,sentiment_neg,sentiment_neu,sentiment_pos
0,2019-08-31 23:56:24+00:00,"0,122","0,385","0,494"
1,2020-11-12 11:05:43+00:00,"0,0","0,85","0,15"
2,2019-08-31 23:54:13+00:00,"0,0","1,0","0,0"
3,2019-08-31 23:54:07+00:00,NaN,NaN,NaN
4,2019-08-31 23:53:09+00:00,"0,0","0,719","0,281"
...,...,...,...,...
10364,2019-08-01 01:12:15+00:00,"0,046","0,883","0,071"
10365,2019-08-01 01:06:09+00:00,"0,209","0,67","0,121"
10366,2019-08-01 01:04:32+00:00,NaN,NaN,NaN
10367,2019-08-01 00:37:30+00:00,"0,0","0,863","0,137"


In [14]:
name = "YouGov_.csv"
test,test1 = name.split('_')
test, test1

('YouGov', '.csv')

In [9]:
path = r"../tab2/new_tweets"
# folder_names = os.listdir(path)
# folder_names
files = glob.glob(os.path.join(path + "/*", "*.csv")) 
files
# df = pd.concat([pd.read_csv(f, sep=";") for f in files])

['../tab2/new_tweets\\Airbnb\\Airbnb_tweets.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_count_daily.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_count_weekly.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_sentiment_day_percent.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_sentiment_month_percent.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_sentiment_week_percent.csv']

In [13]:
path = r"../tab2/new_tweets"
files = glob.glob(os.path.join(path, "**", "*.csv"), recursive=True)
files

['../tab2/new_tweets\\Airbnb\\Airbnb_tweets.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_count_daily.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_count_weekly.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_sentiment_day_percent.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_sentiment_month_percent.csv',
 '../tab2/new_tweets\\Airbnb\\Airbnb_twitter_sentiment_week_percent.csv',
 '../tab2/new_tweets\\Nike\\Nike_tweets.csv',
 '../tab2/new_tweets\\Nike\\Nike_twitter_count_daily.csv',
 '../tab2/new_tweets\\Nike\\Nike_twitter_count_weekly.csv',
 '../tab2/new_tweets\\Nike\\Nike_twitter_sentiment_day_percent.csv',
 '../tab2/new_tweets\\Nike\\Nike_twitter_sentiment_month_percent.csv',
 '../tab2/new_tweets\\Nike\\Nike_twitter_sentiment_week_percent.csv']

In [68]:
path = r"../tab2/new_tweets" if data_source == "tweets" else r"../tab2/new_stocktwits"
files = glob.glob(os.path.join(path, "**", "*.csv"), recursive=True)

# Define a dictionary to store the concatenated DataFrames for each brand and data type
data_type_dict = {}
exclude = ["tweets", "stocktwits"]

for file in files:
    # Extract the brand name and data type from the file path
    file_name = os.path.splitext(os.path.basename(file))[0]
    data_type = file_name.split("_", 1)[1]
    
    if data_type not in data_type_dict and data_type not in exclude:
        data_type_dict[data_type] = pd.read_csv(file)
    elif data_type not in exclude:
        data_type_dict[data_type] = pd.concat([data_type_dict[data_type], pd.read_csv(file)])

data_type_dict


{'twitter_count_daily':      brand        Date  tweet_count
 0   Airbnb  2018-08-13            5
 1   Airbnb  2018-08-16            4
 2   Airbnb  2018-11-15            2
 3   Airbnb  2019-04-25            2
 4   Airbnb  2019-05-01            1
 ..     ...         ...          ...
 36    Nike  2019-07-31          368
 37    Nike  2019-08-07            1
 38    Nike  2020-02-06            1
 39    Nike  2020-11-12            1
 40    Nike  2020-11-17            2
 
 [115 rows x 3 columns],
 'twitter_count_weekly':      brand        Date  tweet_count
 0   Airbnb  2018-08-13            9
 1   Airbnb  2018-11-12            2
 2   Airbnb  2019-04-22            2
 3   Airbnb  2019-04-29            4
 4   Airbnb  2019-07-01         2224
 5   Airbnb  2019-07-08         2402
 6   Airbnb  2019-07-15         2326
 7   Airbnb  2019-07-22         2458
 8   Airbnb  2019-07-29         2568
 9   Airbnb  2019-08-05         2346
 10  Airbnb  2019-08-12         2209
 11  Airbnb  2019-08-19         2354
 

In [71]:
data_type_dict.keys()

dict_keys(['twitter_count_daily', 'twitter_count_weekly', 'twitter_sentiment_day_percent', 'twitter_sentiment_month_percent', 'twitter_sentiment_week_percent'])

In [47]:
# # Parse the CSV data into a list of rows
# csv_rows = list(csv.reader(decoded_str.splitlines()))

# # Write the contents of the CSV data to the specified directory
# with open(os.path.join(directory, name), 'a', newline='') as fp:
#     writer = csv.writer(fp)
#     writer.writerows(csv_rows)

,date,sentiment_neg,sentiment_neu,sentiment_pos,brand
0,2019-07-31 23:54:53+00:00,"0,0","0,629","0,371",Nike
1,2020-11-12 11:05:43+00:00,"0,0","0,85","0,15",Nike
2,2019-07-31 23:54:13+00:00,NaN,NaN,NaN,Nike
3,2019-07-31 23:41:49+00:00,NaN,NaN,NaN,Nike
4,2019-07-31 23:15:04+00:00,"0,059","0,838","0,103",Nike
...,...,...,...,...,...
10686,2019-07-01 00:17:00+00:00,"0,0","0,855","0,145",Nike
10687,2019-07-01 00:13:09+00:00,"0,0","1,0","0,0",Nike
10688,2019-07-01 00:09:09+00:00,"0,0","1,0","0,0",Nike
10689,2019-07-01 00:07:55+00:00,"0,0","1,0","0,0",Nike


In [33]:
os.path.split(os.path.splitext(file)[0])

('../tab2/new_tweets\\Nike', 'Nike_twitter_sentiment_week_percent')

In [39]:
file_name = os.path.splitext(os.path.basename(file))[0]
file_name
brand_name = file_name.split("_", 1)[0]
data_type = file_name.split("_", 1)[1]
data_type

'twitter_sentiment_week_percent'

In [41]:
brand_name, data_type = file_name.split("_", 1)
data_type

'twitter_sentiment_week_percent'